<a href="https://colab.research.google.com/github/goltango/ArqComp_ej10_TP1_ALU/blob/master/codigo_pfa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#instalamos prerequisitos
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 9.5 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from random import randint
import shap

In [4]:
# LLamas la superbase
# Cargar el archivo CSV sin analizar fechas
df = pd.read_csv('Superbase04-07.csv')

# Convertir la columna 'fecha' a formato de fecha eficiente de python
df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d')

# Establecer la columna 'fecha' como índice
df.set_index('fecha', inplace=True)

In [ ]:
df.head()

,cl,mm,mm3,mm6,mm9,mm12,mm15,Tsup,Temp_agro,Viento_agro,...,-5.8,6.4,7.4,8.4,9.4,10.4,11.4,12.4,wind_speed,air_temp
fecha,,,,,,,,,,,,,,,,,,,,,
2001-01-01,46.1138,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-02,28.2610,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-03,47.4535,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-04,193.7262,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-05,11.4263,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Se incian listas vacias de nombres cols1,cols2,names1,names2
cols1, cols2 = list(), list()
names1,names2=list(), list()

In [6]:
# Se extraen las primeras 10 columnas del Dataframe y se guardan en "Cols1". Los nombres de estas columnas se guardan en "names1".
cols1.append(df[df.columns[:10]])
names1+=df.columns[:10].to_list()

In [7]:
print(cols1)

[                  cl   mm  mm3  mm6  mm9  mm12  mm15  Tsup  Temp_agro  \
fecha                                                                   
2001-01-01   46.1138  0.0  NaN  NaN  NaN   NaN   NaN   NaN        NaN   
2001-01-02   28.2610  0.0  NaN  NaN  NaN   NaN   NaN   NaN        NaN   
2001-01-03   47.4535  5.0  NaN  NaN  NaN   NaN   NaN   NaN        NaN   
2001-01-04  193.7262  0.0  5.0  NaN  NaN   NaN   NaN   NaN        NaN   
2001-01-05   11.4263  0.0  5.0  NaN  NaN   NaN   NaN   NaN        NaN   
...              ...  ...  ...  ...  ...   ...   ...   ...        ...   
2024-05-05    6.9704  NaN  NaN  NaN  NaN   NaN   NaN   NaN        NaN   
2024-05-06   53.3345  NaN  NaN  NaN  NaN   NaN   NaN   NaN        NaN   
2024-05-07   54.7198  NaN  NaN  NaN  NaN   NaN   NaN   NaN        NaN   
2024-05-08    5.3878  NaN  NaN  NaN  NaN   NaN   NaN   NaN        NaN   
2024-05-09   67.0280  NaN  NaN  NaN  NaN   NaN   NaN   NaN        NaN   

            Viento_agro  
fecha                  

In [8]:
#Se agrega a "cols1" que contenía las 10 primeras columnas del Datframe una versión desplazada en el tiempo de las columnas 8 a la 10
#Corresponde en el Datframe original que era superbaseffrN (Hay que cambiarlo por el actual) a las columnas Tsup(Modelada Joni), Temp agro, Viento agro(Cosquín agricultura-OMIXOM) y cl-a(Alba).
#Es decir, de esas variables se agregan las columnas correpondientes a 1 a 5 días hacia atrás en el tiempo

for i in range(5):
    cols1.append(df[['cl']+list(df.columns[7:10])].shift(i+1))
    names1+=[df.columns[j]+'('+str(-(i+1))+')' for j in [0,7,8,9]]

In [9]:
#Se toma la primera columna del dataframe, en este caso es cl-a. A "cols2" se agrega una version desplazada en el tiempo de cl-a pero esta vez hacia adelante
#La iteracion se realiza 10 días hacia adelante. Los valores de la columna se guardan en "cols2" y los nombres en "names2"

for i in range(10):
    cols2.append(df[df.columns[:1]].shift(-(i+1)))
    names2+=['cl('+str(i+1)+')']

#Agrego a "cols1" y "names1" las restantes variables que me quedaron a partir de la columna 10 en mi Dataframe original. Luego concateno todo en agg
cols1.append(df[df.columns[10:]])
names1+=df.columns[10:].to_list()
cols=cols1+cols2
names1+=names2
agg = pd.concat(cols, axis=1)
agg.columns=names1

In [10]:
# split dataset into input and output columns
agg.dropna(subset=['cl(1)', 'cl(2)', 'cl(3)', 'cl(4)', 'cl(5)', 'cl(6)', 'cl(7)', 'cl(8)', 'cl(9)', 'cl(10)'],inplace=True)# aca eloimino los dias que no tengo dato de clorofila

data = agg.values
X, y = data[:, :-10], data[:, -10:]# acá tengo inputs y outputs

In [11]:
def entrenador(semillasplit,train_X, test_X, train_y, test_y,names1):
    # defino los diccionarios donde voy a guardar las cosas
    modelos={}
    explainer={}
    shap_values={}
    # en la primer key correspondiente al modelo guardo los nombres de los inputs
    modelos['Importancia con semilla: '+str(semillasplit)]=pd.DataFrame({'Variable': names1[:-10]})
    # guardo el y y defino el yhat para construir luego el validador
    yhat=pd.DataFrame()
    y=pd.DataFrame(test_y)
    # hago los modelos 10 dias hacia adelante o sea tengo que hacer 10 modelos
    for i in range(10):
        # define model y lo guardo en el diccionario en la key "modelo tal con semilla cual" donde tal es la cantidad de dias hacia adelante y cual es la semilla que le haya dado
        modelos['modelo'+str(i+1)+' con semilla: '+str(semillasplit)]=xgb.XGBRegressor(n_estimators=5000, max_depth=3, eta=.1, subsample=.5, colsample_bytree=.7)#
        # fit model
        modelos['modelo'+str(i+1)+' con semilla: '+str(semillasplit)].fit(train_X, train_y[:,i])
        # guardo la importancia de las variables que me da el xgb (que supongo que no voy a usar porque voy a usar shap)
        #modelos['Importancia con semilla: '+str(semillasplit)]['Importancia'+str(i+1)]= modelos['modelo'+str(i+1)+' con semilla: '+str(semillasplit)].feature_importances_
        #asigno los pronósticos al yhat que habia definido que con el y van a formar el validador
        yhat['predicción '+str(i+1)] = modelos['modelo'+str(i+1)+' con semilla: '+str(semillasplit)].predict(test_X)
        # SHAP values
        #explainer['Explicador'+str(i+1)+' con semilla: '+str(semillasplit)] = shap.Explainer(modelos['modelo'+str(i+1)+' con semilla: '+str(semillasplit)], train_X)
        #selecciono solo los valores para tenerlos mas a mano
        #shap_values['Valoresdeimporetancia'+str(i+1)+' con semilla: '+str(semillasplit)] = explainer['Explicador'+str(i+1)+' con semilla: '+str(semillasplit)](test_X)
    validador=pd.concat([y,yhat],axis=1)
    #validador.to_csv('yhatdel3ffr.csv')
    nombre1,nombre2=list(), list()
    nombre1+=['y'+str(j) for j in range(10)]
    nombre2+=['yhat'+str(j) for j in range(10)]
    validador.columns=nombre1+nombre2
    return([modelos,validador,explainer,shap_values])

In [16]:
import json

# Función para guardar semillas en un archivo JSON
def guardar_semillas(semillas):
    filename=f'semillas_generadas_{str(len(semillas))}.json'
    with open(filename, 'w') as f:
        json.dump(semillas, f)
        print("Se guardaron " + str(len(semillas)) + " semillas.")

# Función para cargar semillas desde un archivo JSON
def cargar_semillas(filename='semillas.json'):
    if os.path.exists(filename) and os.path.getsize(filename) > 0:
        with open(filename, 'r') as f:
            semillas = json.load(f)
            print("Se cargaron " + str(len(semillas)) + " semillas.")
        return semillas
    else:
        return []

In [13]:
import pickle
def cargar_estado(num_iteraciones, block_size):
    global registro
    for i in range(0, num_iteraciones, block_size):
        filename = f'estado_intermedio_{i // block_size}.pkl'
        if os.path.exists(filename):
            with open(filename, 'rb') as f:
                print(f'Cargando estado desde {filename}...')
                partial_registro = pickle.load(f)
                registro.update(partial_registro)

def guardar_estado(parte, block_size):
    start_index = parte * block_size
    end_index = start_index + block_size
    partial_registro = {k: v for i, (k, v) in enumerate(registro.items()) if start_index <= i < end_index}
    with open(f'estado_intermedio_{parte}.pkl', 'wb') as f:
        print(f'Guardando estado en estado_intermedio_{parte}.pkl...')
        pickle.dump(partial_registro, f)

In [ ]:
import time
from datetime import timedelta
import os

num_iteraciones = 20  # Número total de iteraciones
block_size = 10  # Por ejemplo, guardamos cada x iteraciones

# Generar o cargar todas las semillas antes de comenzar las iteraciones
semillas = cargar_semillas()
if not semillas:
    print('Generando semillas...')
    semillas = [randint(1, 5000) for _ in range(num_iteraciones)]
    guardar_semillas(semillas)

# Generar un diccionario para almacenar los resultados
registro={}

# Definir el rango de iteraciones que deseas ejecutar
start_iter = 0  # Primera iteración a ejecutar
end_iter = 20    # Última iteración a ejecutar
print('Iteraciones a ejecutar: ' + str(end_iter - start_iter))
print('Desde la iteracion ' + str(start_iter) + ' hasta la iteracion ' + str(end_iter-1))

# Cargar estado previo si existe
cargar_estado(num_iteraciones, block_size)

# Se toma el tiempo de inicio de la corrida
start = time.time()
for i in range(start_iter, end_iter):
    semillasplit = semillas[i]
    print('Iteracion: '+ str(i) + ' Semilla: '+str(semillasplit) + ' Running...')
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=.2, random_state=semillasplit)#)
    aux=entrenador(semillasplit,train_X, test_X, train_y, test_y,names1)
    registro['Modelos con Semilla'+str(semillasplit)]=aux[0]
    registro['Validador con Semilla'+str(semillasplit)]=aux[1]
    registro['Explicador con Semilla'+str(semillasplit)]=aux[2]
    registro['valores con Semilla'+str(semillasplit)]=aux[3]
    enlapsed_time = time.time()
    print('Enlapsed time: ' + str(timedelta(seconds=enlapsed_time - start)))
    # Guardar el estado cada block_size iteraciones
    if (i + 1) % block_size == 0:
        guardar_estado(i // block_size, block_size)

# Guardar el estado final si no es múltiplo de block_size
if end_iter % block_size != 0:
    guardar_estado(end_iter // block_size, block_size)

Se cargaron 20 semillas.
Iteraciones a ejecutar: 20
Desde la iteracion 0 hasta la iteracion 19
Iteracion: 0 Semilla: 1098 Running...
Enlapsed time: 0:06:56.816097
Iteracion: 1 Semilla: 2377 Running...


In [ ]:
registro.keys()

dict_keys(['Modelos con Semilla2599', 'Validador con Semilla2599', 'Explicador con Semilla2599', 'valores con Semilla2599', 'Modelos con Semilla1167', 'Validador con Semilla1167', 'Explicador con Semilla1167', 'valores con Semilla1167', 'Modelos con Semilla4832', 'Validador con Semilla4832', 'Explicador con Semilla4832', 'valores con Semilla4832', 'Modelos con Semilla1745', 'Validador con Semilla1745', 'Explicador con Semilla1745', 'valores con Semilla1745', 'Modelos con Semilla2455', 'Validador con Semilla2455', 'Explicador con Semilla2455', 'valores con Semilla2455', 'Modelos con Semilla2022', 'Validador con Semilla2022', 'Explicador con Semilla2022', 'valores con Semilla2022', 'Modelos con Semilla1824', 'Validador con Semilla1824', 'Explicador con Semilla1824', 'valores con Semilla1824', 'Modelos con Semilla682', 'Validador con Semilla682', 'Explicador con Semilla682', 'valores con Semilla682', 'Modelos con Semilla4298', 'Validador con Semilla4298', 'Explicador con Semilla4298', 'va

In [ ]:
# prompt: show data about first item of the registro

print(registro['Modelos con Semilla2599'])
print(registro['Validador con Semilla2599'])
print(registro['Explicador con Semilla2599'])
print(registro['valores con Semilla2599'])

{'Importancia con semilla: 2599':       Variable  Importancia1  Importancia2  Importancia3  Importancia4  \
0           cl      0.006393      0.007526      0.006389      0.005179   
1           mm      0.007280      0.012563      0.006688      0.006589   
2          mm3      0.008185      0.008167      0.008469      0.006134   
3          mm6      0.007735      0.009852      0.008215      0.007769   
4          mm9      0.007938      0.007532      0.007237      0.006939   
..         ...           ...           ...           ...           ...   
95        10.4      0.015025      0.033386      0.006156      0.005094   
96        11.4      0.005622      0.007410      0.006738      0.015098   
97        12.4      0.008187      0.014581      0.021771      0.010565   
98  wind_speed      0.007020      0.016485      0.012956      0.008511   
99    air_temp      0.014758      0.025095      0.015262      0.008300   

    Importancia5  Importancia6  Importancia7  Importancia8  Importancia9  \
0

In [ ]:


# Guardar el diccionario en un archivo usando pickle
with open('registro_10iterations.pkl', 'wb') as file:
    pickle.dump(registro, file)